## Предсказание траекторий автомобилей

In [ ]:
from IPython.display import Video

In [ ]:
Video("motion-prediction-video.mp4", width=400, height=400)

## Соревнование https://research.yandex.com/shifts/vehicle-motion-prediction

Трек по предсказанию траекторий движения автомобилей. Соревнование в целом посвящено исследованию подходов к оценке неопределенности и устойчивости моделей к сдвигам во входных данных.

![Ансамбль моделей](uncertainty.jpeg)

## Что представляют из себя данные

Единицей данных в датасете является сцена. Сцена содержит в себе информацию о динамических объектах (автомобилях и пешеходах), векторную карту дорожного графа, состояния светофоров, теги для сцены и отдельных агентов.

Данные в сцене разбиты на две зоны: прошлое и будущее. Для каждого автомобиля нам известны следующие параметры:
- уникальный идентификатор
- положение в глобальной системе координат
- размеры
- скорость
- ускорение

Аналогичные данные известны для самого беспилотника. Для пешехедов данные ограничиваются id, положением, размером и скоростью.

Дорожный граф содержит в себе информацию о полосах, полигон дороги, полигоны пешеходных переходов.

Ссылка на тест сет, с которым поработаем на семинаре: https://disk.yandex.ru/d/M_4ED0r19OnSrg

## API для работы с датасетом

Рекомендую установить в virtualenv, чтобы не ставить лишние пакеты в хостовую систему.
```
git clone git@github.com:yandex-research/shifts.git
cd shifts/sdc
pip install .
```

## Посмотрим на данные

Исходные сырые данные хранятся в формате protubuf (https://developers.google.com/protocol-buffers), он позволяет удобно хранить и рабоать со структурированными объектами.

In [ ]:
from ysdc_dataset_api.utils import read_scene_from_file

In [ ]:
scene = read_scene_from_file('/Users/gerrok/test/011/011000.pb')

In [ ]:
print(f'Тип объекта: {type(scene)}')
print(f'Поля объекта: {[field for field in dir(scene) if not field.startswith("_") and field.islower()]}')

Каждая сцена содержит в себе данные о прошлом и будущем на 5 секунд (всего 10 секунд). Это время разбито на 50 дискретных таймстемпов с частотой  5Hz.

In [ ]:
print(f'Горизонт прошлого: {len(scene.past_vehicle_tracks)}, горизонт будущего: {len(scene.future_vehicle_tracks)}')

Индекс 0 в past_tracks соответсвтует -5 секундам в истории, индекс 24 -- нулевая секунда, момент предсказния.

In [ ]:
print(f'Количество машин в момент предсказания: {len(scene.past_vehicle_tracks[-1].tracks)}')
print('Информация об одной из машин в момент предсказания:')
print(scene.past_vehicle_tracks[-1].tracks[0])

В сцене есть поле `prediction_requests` содержащее id автомобилей, для которых необходимо сделать предсказание. Помимо этого реквесты помечены тегами, описывающими характер движения автомобиля. Полный список тегов можно найти в файле `tags.proto` [ссылка](https://github.com/yandex-research/shifts/blob/main/sdc/ysdc_dataset_api/proto/tags.proto)

In [ ]:
for r in scene.prediction_requests:
    if r.track_id == scene.past_vehicle_tracks[-1].tracks[0].track_id:
        print(r)

Тэги есть так же и у сцен. Они описывают различные срезы данных, по которым проходило разбиение в соревновании.

In [ ]:
print(scene.scene_tags)

Давайте нарисуем уже сцену и посмотрим, как оно выглядит не в скучных числах.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import yaml
from matplotlib import collections as mc

from ysdc_dataset_api.dataset import MotionPredictionDataset
from ysdc_dataset_api.features import FeatureRenderer
from ysdc_dataset_api.utils import transform_2d_points

In [ ]:
with open('renderer_config.yaml') as f:
    renderer_config = yaml.safe_load(f)

In [ ]:
renderer = FeatureRenderer(renderer_config)

In [ ]:
dataset_path = '/Users/gerrok/test/'

In [ ]:
dataset = MotionPredictionDataset(
    dataset_path=dataset_path,
    feature_producers=[renderer],
    transform_ground_truth_to_agent_frame=True)

In [ ]:
print(f'Количество сцен в датасете: {dataset.num_scenes}')

In [ ]:
dataset_iter = iter(dataset)

# Проитерируемся по датасету в поисках машины, проехавшей более 2 метров по одной из координат.
while True:
    data_item = next(dataset_iter)
    if data_item['ground_truth_trajectory'][-1, 0] > 2.0 or data_item['ground_truth_trajectory'][-1, 1] > 2.0:
        break

In [ ]:
print(f'Датасет для каждого объекта возвращает набор следующих полей: {[k for k in data_item.keys()]}')

In [ ]:
print(f'Размеры фичемап, которые нам отдал рендерер: {data_item["feature_maps"].shape}')

In [ ]:
# Plot vehicles occupancy, pedestrian occupancy, lane occupancy and road polygon
plt.figure(figsize=(10, 10))
plt.imshow(data_item['feature_maps'][0], origin='lower', cmap='binary', alpha=0.7)
plt.imshow(data_item['feature_maps'][6], origin='lower', cmap='binary', alpha=0.5)
plt.imshow(data_item['feature_maps'][13], origin='lower', cmap='binary', alpha=0.2)
plt.imshow(data_item['feature_maps'][16], origin='lower', cmap='binary', alpha=0.1)

# Переведем ground truth траекторию агента в систему координат фичемапы
transformed_gt = transform_2d_points(data_item['ground_truth_trajectory'], renderer.to_feature_map_tf)
transformed_gt = np.round(transformed_gt - 0.5).astype(np.int32)

ax = plt.gca()
ax.add_collection(mc.LineCollection([transformed_gt], color='green'))

## Метрики

Одними из общепринятых метрик для оценки качества предсказания траекторий являются Average Displacement Error и Final Displacement Error, а так же их модификации minADE@k, minFDE@k:
- ADE - среднее по таймстемпам L2 отклонеие предсказания от ground truth
- FDE - L2 отклонение последней предсказанной точки траектории от ground truth
- minADE@k -- минимальное значение ADE по k наиболее вероятных предсказанных гипотез
- minFDE@k -- минимальное значение FDE по k наиболее вероятных предсказанных гипотез

In [ ]:
def ade(y_true, y_pred):
    """
    Insert your code for ADE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

In [ ]:
def fde(y_true, y_pred):
    """
    Insert your code for FDE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

In [ ]:
def min_ade(y_true, y_pred):
    """
    Insert your code for minADE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_modes, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

In [ ]:
def min_fde(y_true, y_pred):
    """
    Insert your code for minFDE computation below.
    
    Args:
        y_true (np.ndarray): shape (batch, n_timestamps, 2)
        y_pred (np.ndarray): shape (batch, n_modes, n_timestamps, 2)

    Returns:
        np.ndarray: shape (batch, 1)
    """

## Модель с константным предсказанием

In [ ]:
import torch
import tqdm

from ysdc_dataset_api.features import FeatureVectorizer

In [ ]:
class BaselineModel(torch.nn.Module):
    def __init__(self, gt_time_grid):
        super().__init__()
        self._gt_time_grid = torch.tensor(gt_time_grid)
    
    def forward(self, velocity):
        states = torch.einsum('bc,t->btc', velocity, self._gt_time_grid)
        return states

In [ ]:
model = BaselineModel(np.linspace(0.2, 5, 25))

In [ ]:
with open('vectorizer_config.yaml') as f:
    vectorizer_config = yaml.safe_load(f)
vectorizer = FeatureVectorizer(vectorizer_config)

dataset = MotionPredictionDataset(
    dataset_path='/Users/gerrok/test/',
    prerendered_dataset_path='/Users/gerrok/test/',
    feature_producers=[vectorizer],
    transform_ground_truth_to_agent_frame=True,
)

In [ ]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    num_workers=8,
)

In [ ]:
ades = []
fdes = []
for batch in tqdm.tqdm(dataloader):
    # Возьмём скорость по (x, y) в последний известный момент времени
    predictions = model(batch['vector_features'][:, -1, 2:4])
    ades.append(ade(batch['ground_truth_trajectory'], predictions))
    fdes.append(fde(batch['ground_truth_trajectory'], predictions))

In [ ]:
np.mean(np.concatenate(ades)), np.mean(np.concatenate(fdes))

## Визуализация предсказаний

## Мультимодальность

## Задание

В качестве задания предлагается обучить нейросетку для предсказания траекторий и побить наш бейзлайн.
Ссылки на данные:
- [трейн]()
- [валидация](https://disk.yandex.ru/d/3Lu6_6BgwkXlgw)
- [тест](https://disk.yandex.ru/d/M_4ED0r19OnSrg)

В своей реализации вы можете использовать векторные фичи для объекта, а так же растровые картинки.


Критерии оценки (subject to change):
- сделать нейросетку, которая бьёт бейзлайн (4 балла)
- использовать картиночные фичи (4 балла)
- научить сеть предсказывать более одной моды (2 балла)

In [ ]:
# Для использования картиночных фичей можно использовать уже нарендеренные картинки (они есть в датасете).
# Рендерить с нуля может быть довольно долго.
# Пример использования готовых картинок в датасете:

# dataset = MotionPredictionDataset(
#     dataset_path='/Users/gerrok/test/',
#     prerendered_dataset_path='/Users/gerrok/test/',  # Тут мы указываем путь, где искать готовые картинки
#     feature_producers=[vectorizer],
#     transform_ground_truth_to_agent_frame=True,
# )